In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
#from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import random

#from ggplot import *
#from scipy.stats import logistic
#from sklearn.metrics import roc_curve
#from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import confusion_matrix, mean_squared_error
#from sklearn.model_selection import train_test_split
#from sklearn import preprocessing
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import *
#from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
#import graphviz
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.svm import SVC, SVR

# 한글 폰트 실행
from matplotlib import rcParams
from matplotlib import font_manager,rc
rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='NanumGothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)


In [3]:
import time

def execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Execution time for {func.__name__}: {int(hours)}h {int(minutes)}m {seconds:.2f}s")
        return result
    return wrapper


@execution_time
def clean_df(data):
    data['고객ID'] = data['고객ID'].astype(str)
    data = reduce_mem_usage(data)
    print(data.info())
    return data


import pandas as pd 
import numpy as np

def reduce_mem_usage(df, float16_as32=True):
    # memory_usage()는 데이터프레임의 각 열의 메모리 사용량을 계산하고, sum은 그 값을 합산합니다. B -> KB -> MB로 변환합니다.
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터프레임의 초기 메모리 사용량은 {:.2f} MB입니다.'.format(start_mem))

    for col in df.columns:  # 각 열의 이름을 반복합니다.
        col_type = df[col].dtype  # 열의 데이터 타입을 가져옵니다.
        if col_type != object and str(col_type) != 'category':  # 객체 타입이 아닌 수치형 변수만 처리합니다.
            c_min, c_max = df[col].min(), df[col].max()  # 해당 열의 최소값과 최대값을 계산합니다.
            if str(col_type)[:3] == 'int':  # 정수형 변수일 경우 (int8, int16, int32, int64 모두 포함)
                # 이 열의 값 범위가 int8의 범위에 있을 경우 타입을 int8로 변환합니다. (-128부터 127까지)
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                # 이 열의 값 범위가 int16의 범위에 있을 경우 타입을 int16으로 변환합니다. (-32,768부터 32,767까지)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                # 이 열의 값 범위가 int32의 범위에 있을 경우 타입을 int32로 변환합니다. (-2,147,483,648부터 2,147,483,647까지)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                # 이 열의 값 범위가 int64의 범위에 있을 경우 타입을 int64로 변환합니다. (-9,223,372,036,854,775,808부터 9,223,372,036,854,775,807까지)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:  # 실수형일 경우
                # 값이 float16의 범위에 있을 경우 타입을 변환합니다. 더 높은 정밀도가 필요하면 float32를 고려합니다.
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:  # 더 높은 정밀도가 필요하면 float32를 선택할 수 있습니다.
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)
                # 값이 float32의 범위에 있을 경우 타입을 float32로 변환합니다.
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                # 값이 float64의 범위에 있을 경우 타입을 float64로 변환합니다.
                else:
                    df[col] = df[col].astype(np.float64)
    # 최종 메모리 사용량을 계산합니다.
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 데이터프레임의 메모리 사용량은: {:.2f} MB입니다.'.format(end_mem))
    # 초기 메모리 사용량과 비교하여 몇 퍼센트 감소했는지 계산합니다.
    print('메모리 사용량이 {:.1f}% 감소했습니다.'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [4]:
import pandas as pd
DATA_PATH = './dataset/CARD/'
DATA_PATH_2 = './dataset/CUSTOMER/'
DATA_PATH_3 = './dataset/CC/'
cc_22_40 = pd.read_csv(DATA_PATH_3 + 'cc_22_40.csv', encoding="utf-8-sig")
cc_22_50 = pd.read_csv(DATA_PATH_3 + 'cc_22_50.csv', encoding="utf-8-sig")
cc_22_60 = pd.read_csv(DATA_PATH_3 + 'cc_22_60.csv', encoding="utf-8-sig")

In [5]:
customer_22_40 = pd.read_csv(DATA_PATH_2 + 'customer_22_40.csv', encoding="utf-8-sig")

In [6]:
cc_22_40.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14076734 entries, 0 to 14076733
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      int64 
 1   거래년월      int64 
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int64 
dtypes: int64(3), object(4)
memory usage: 751.8+ MB


In [7]:
cc_22_40_raw = pd.read_csv(DATA_PATH_3 + 'cc_22_40(raw).csv', encoding="utf-8-sig")
cc_22_50_raw = pd.read_csv(DATA_PATH_3 + 'cc_22_50(raw).csv', encoding="utf-8-sig")
cc_22_60_raw = pd.read_csv(DATA_PATH_3 + 'cc_22_60(raw).csv', encoding="utf-8-sig")

In [8]:
cc_22_40.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14076734 entries, 0 to 14076733
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      int64 
 1   거래년월      int64 
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int64 
dtypes: int64(3), object(4)
memory usage: 751.8+ MB


In [9]:
cc_22_40_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14076734 entries, 0 to 14076733
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      int64 
 1   거래년월      int64 
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int64 
dtypes: int64(3), object(4)
memory usage: 751.8+ MB


In [10]:
cc_22_40.head()

,고객ID,거래년월,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,9997364303,202212,이동통신요금,서울특별시,중구,5번이하,70000
1,9997364303,202212,외식,경상북도,영천시,5번이하,120000
2,9997364303,202212,온라인쇼핑,경기도,성남시 분당구,5번이하,80000
3,9997364303,202212,의료/보험,경상북도,영천시,5번이하,10000
4,9997364303,202212,외식,경상북도,영천시,5번이하,30000


In [11]:
cc_22_40_raw.head()

,고객ID,거래년월,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,9997364303,202212,이동통신요금,서울특별시,중구,5번이하,70000
1,9997364303,202212,일식·회집,경상북도,영천시,5번이하,120000
2,9997364303,202212,인터넷종합Mall,경기도,성남시 분당구,5번이하,80000
3,9997364303,202212,의 원,경상북도,영천시,5번이하,10000
4,9997364303,202212,일반한식,경상북도,영천시,5번이하,30000


In [14]:
cc_22_40_raw['가맹점업종명'].unique()

array(['이동통신요금', '일식·회집', '인터넷종합Mall', '의 원', '일반한식', '인터넷P/G', '문구용품',
       '스넥', '조세서비스', '편 의 점', '서양음식', '대형할인점', '치과의원', '슈퍼마켓', '기타음료식품',
       '자사카드발행백화점', '손해보험', '약 국', 'SK주유소', '스포츠의류', '농·축·수산품', '콘도',
       '주유소', '화원', '연 쇄 점', '세탁소', '치과병원', '중국식', '농,축협직영매장', '일반서적',
       '통신판매업1', '생명보험', '기타 전문점', 'GS주유소', '주점', '현대정유(오일뱅크)', '자동차정비',
       '미용원', '스포츠·레져용품', 'SK가스충전소', '업종미등록', '택시', '기타잡화', '철도', '화장품',
       '가전제품', '갈비전문점', '제과점', '기계공구', '이용원', '한 의 원', '상 품 권', '안경',
       '주차장', '유흥주점', '골프경기장', 'E1가스충전소', '스크린골프', 'LPG취급점', '기타숙박업',
       '정육점', '내의판매', '쌍용S-OIL', '당구장', '시 계', '출판 및 인쇄물', '미용재료', '정장',
       '의료용품', '귀금속', '노 래 방', '애완동물', '영화관', 'CATV홈쇼핑', '동물병원', '가례서비스업',
       '세차장', 'DP&E', '농협하나로클럽', '종합병원', '병 원(응급실운영)', '공공요금대행서비스/소득공제대상',
       '기타레져업소', 'PG상품권', '골프연습장', '액세서리', '면 세 점', '기타 교육기관',
       '현대정유가스충전소', '인터넷Mall', '유아원', '고속버스', '기타전기제품', '비영리/비대상',
       '볼 링 장', '비씨카드 정산용(할인)', '예·체능계학원', '보습학원', '외국어학원', '골프용품 전문점',
  

In [15]:
cc_22_40['가맹점업종명'].unique()

array(['이동통신요금', '외식', '온라인쇼핑', '의료/보험', '간편결제', '기타', '음식료품', '편의점',
       '대형마트', '카페/베이커리', '백화점', '주유', '스포츠/레저', '여행/교통', '미용', '반려동물',
       '문화생활', '홈쇼핑', '교육/학원'], dtype=object)

In [16]:
cc_22_50['가맹점업종명'].unique()

array(['외식', '대형마트', '의료/보험', '편의점', '음식료품', '여행/교통', '간편결제', '온라인쇼핑',
       '기타', '카페/베이커리', '주유', '스포츠/레저', '백화점', '미용', '문화생활', '홈쇼핑',
       '이동통신요금', '교육/학원', '반려동물'], dtype=object)

In [17]:
cc_22_60['가맹점업종명'].unique()

array(['여행/교통', '편의점', '대형마트', '카페/베이커리', '외식', '기타', '음식료품', '주유',
       '의료/보험', '간편결제', '미용', '스포츠/레저', '반려동물', '백화점', '온라인쇼핑', '문화생활',
       '홈쇼핑', '교육/학원', '이동통신요금'], dtype=object)

In [18]:
cc_22_40_raw['가맹점업종명'].unique()

array(['이동통신요금', '일식·회집', '인터넷종합Mall', '의 원', '일반한식', '인터넷P/G', '문구용품',
       '스넥', '조세서비스', '편 의 점', '서양음식', '대형할인점', '치과의원', '슈퍼마켓', '기타음료식품',
       '자사카드발행백화점', '손해보험', '약 국', 'SK주유소', '스포츠의류', '농·축·수산품', '콘도',
       '주유소', '화원', '연 쇄 점', '세탁소', '치과병원', '중국식', '농,축협직영매장', '일반서적',
       '통신판매업1', '생명보험', '기타 전문점', 'GS주유소', '주점', '현대정유(오일뱅크)', '자동차정비',
       '미용원', '스포츠·레져용품', 'SK가스충전소', '업종미등록', '택시', '기타잡화', '철도', '화장품',
       '가전제품', '갈비전문점', '제과점', '기계공구', '이용원', '한 의 원', '상 품 권', '안경',
       '주차장', '유흥주점', '골프경기장', 'E1가스충전소', '스크린골프', 'LPG취급점', '기타숙박업',
       '정육점', '내의판매', '쌍용S-OIL', '당구장', '시 계', '출판 및 인쇄물', '미용재료', '정장',
       '의료용품', '귀금속', '노 래 방', '애완동물', '영화관', 'CATV홈쇼핑', '동물병원', '가례서비스업',
       '세차장', 'DP&E', '농협하나로클럽', '종합병원', '병 원(응급실운영)', '공공요금대행서비스/소득공제대상',
       '기타레져업소', 'PG상품권', '골프연습장', '액세서리', '면 세 점', '기타 교육기관',
       '현대정유가스충전소', '인터넷Mall', '유아원', '고속버스', '기타전기제품', '비영리/비대상',
       '볼 링 장', '비씨카드 정산용(할인)', '예·체능계학원', '보습학원', '외국어학원', '골프용품 전문점',
  

In [19]:
cc_22_50_raw['가맹점업종명'].unique()

array(['일반한식', '슈퍼마켓', '종합병원', '편 의 점', '중국식', '스넥', '약 국', '고속버스',
       '인터넷P/G', '인터넷종합Mall', '안경', '택시', '제과점', '비영리/비대상', '농·축·수산품',
       '기타잡화', '일식·회집', '서양음식', '대형할인점', '정육점', '현대정유(오일뱅크)', '골프경기장',
       '스포츠·레져용품', '상 품 권', '골프용품 전문점', '화원', '농협하나로클럽', '기타숙박업', '시 계',
       '주유소', '당구장', '쌍용S-OIL', '사우나', '유흥주점', '의 원', '내의판매', '자사카드발행백화점',
       '주차장', '노 래 방', '주점', '미용원', '2급 호텔', '영화관', '자동차정비', 'GS주유소',
       '기타음료식품', 'CATV홈쇼핑', '1급 호텔', '기타 전문점', '쌍용S-OIL가스충전소', '자동차부품',
       '철도', '스포츠의류', '단란주점', '문구용품', '종합용역', '공공요금대행서비스/소득공제대상', '면 세 점',
       '농,축협직영매장', '이동통신요금', '연 쇄 점', '레져업소', '인터넷Mall', 'SK주유소',
       'E1가스충전소', '병 원(응급실운영)', '비영리/대상', '가정용품수리', '골프연습장',
       '비씨카드 정산용가맹점', '기념품점', '항공사', '관광여행', '예·체능계학원', '업종미등록', '이용원',
       '정장', '화장품', '가 방', '세탁소', '일반서적', '컴퓨터', '침구·수예점', '신 발', '사무서비스',
       '기타레져업소', '가전제품', '기타보험', '공공요금대행서비스/소득공제비대상', '기타전기제품', '농축수산가공품',
       '주방용식기', '조명기구', '민예·공예품', '한 의 원', '치과의원', '미용재료', '화물운송', '액세서리',
 

In [20]:
cc_22_60_raw['가맹점업종명'].unique()

array(['택시', '편 의 점', '슈퍼마켓', '기타음료식품', '서양음식', '고속버스',
       '공공요금대행서비스/소득공제대상', '일반한식', '스넥', 'SK가스충전소', '농,축협직영매장', '철도',
       '정육점', '약 국', '종합병원', '치과병원', '병 원(응급실운영)', '기타잡화', '대형할인점', '제과점',
       '비영리/대상', '의 원', '인터넷P/G', '미용원', '비료,사료,종자', '화장품', '연 쇄 점', '정장',
       '농·축·수산품', '의료용품', '건축용 요업제품', '주유소', '기타유통업', '스포츠·레져용품', '자동차정비',
       '신 발', '목재·석재·철물', '안경', '조명기구', '애완동물', '구내매점(국가기관등)', '손해보험',
       '조세서비스', '자사카드발행백화점', '인터넷종합Mall', '중국식', 'GS주유소', '기계공구', '업종미등록',
       '기타건축자재', '인터넷Mall', '일반서적', 'SK주유소', '쌍용S-OIL', '주방용식기', '미곡상',
       '가전제품', '가 방', '주점', '치과의원', '일식·회집', '기타전기제품', '카페트,커튼,천막,지물',
       '가정용품수리', '화원', '카인테리어', '기타농업관련', '농기계', '사우나', '동물병원', '문구용품',
       '피부미용실', '사무서비스', '주방용구', '공공요금대행서비스/소득공제비대상', '자동차부품', '세탁소',
       '골프경기장', '골프연습장', '영화관', '면 세 점', '기타보험', '스포츠의류', '특급호텔',
       '기타용역서비스', '기타레져업소', '기타건강식품', '침구·수예점', 'DP&E', '자동차시트·타이어',
       '한 의 원', '화물운송', 'CATV홈쇼핑', '유흥주점', '농협하나로클럽', '기타의료기관 및 기타의료기기',
       '민예